In [39]:
def read_text_file(filename: str):
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()
    return content    
    
    
from glob import glob
import os

def read_text_file(filename: str):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            content = file.read()
        return content    

    except:
        print(f'"{filename}" FAILED to read')

def parse_element_data(filename: str):
    contents = read_text_file(filename)

    owner = 'X12'
    release = contents.split('X12 Release')[-1].split()[0].strip()
    name_full, description = contents.split('Share via email')[0].rsplit('\nElements\n', 1)[-1].strip().split('\n')[1:]
    element_id, name_short = name_full.split(' ', 1)
    url = f'https://www.stedi.com/edi/x12-00{release}/element/{element_id}'

    header = contents.split('Stedi.com')[0].split('Share via email')[-1].split('Codes')[0].strip().split('\n')
    data_type = header[1]
    minlen, maxlen = header[-1].replace('Min', '').replace('Max', '').split('/')
    minlen = int(minlen)
    maxlen= int(maxlen)


    #raw_data = contents.split('Stedi.com')[0].split('Codes')[-1]
    raw_data = contents.rsplit('Stedi.com', 1)[0].split('\nCodes\n', 1)[-1].rstrip()

    tabularized = [_.strip().replace('\n', ' - ').split('\t') for _ in raw_data.split('\n ') if _]
    jsonized = {item[0]: item[1] for item in tabularized}

    save_as = f'DE-{element_id}-{release}-X12.json'
    
    meta = {'Filename': save_as,
         'Owner': owner,
         'Release': release,
         'ElementID': element_id, 
         'NameShort': name_short,
         'MinLen': minlen,
         'MaxLen': maxlen,
         'Records': len(tabularized),
         'URL': url
        }

    return {'metadata': meta, 'data':jsonized}, save_as


def process_all_files_in_directory():
    
    files = glob('*.txt')
    for file in files:
        try:
            json_payload, save_as = parse_element_data(file)
            
            with open(save_as, "w") as f:
            #    # indent=4 makes it readable; sort_keys=False keeps your order
                try:
                    json.dump(json_payload, f, indent=4)
                    
                except:
                    print(f'"{file}" FAILED to save')
    
        except:
            print(f'"{file}" FAILED to process')

process_all_files_in_directory()

len(glob('*.txt')) == len(glob('*.txt'))



True